In [35]:
#! /usr/bin/env python3

import glob
import json
import sys

import pandas as pd


files = glob.glob("./metrics/*_weekly.csv")
data_list = []

for path in files:
    # read csv, dump to json
    df = pd.read_csv(path).set_index("task_run__finish_time")
    df.rename(columns={"task_run__finish_time": "Date"}, inplace=True)
    data_dict = df.to_dict()
    data_dict["Name"] = path

    # convert nested dictionaries to lists
    data = {}
    for key, value in data_dict.items():    
        if isinstance(data_dict[key], dict):
            data[key] = []
            for key_, value_ in value.items():
                data[key].append([key_, value_])
                
            data[key] = sorted(data[key], key=lambda x: x[0])
            
            data[key + "Max"] = max(map(lambda x: x[1], data[key]))
            data[key + "Min"] = min(map(lambda x: x[1], data[key]))
        else:
            data[key] = value

    data_list.append(data)
    
# dump list as json
with open("data.json", "w") as out_json:
    json.dump(data_list, out_json)